In [ ]:
import ipywidgets as widgets
from IPython.display import display
import os

from src.template_loader import *
from src.customization import *
from src.runner import *

name_selector = widgets.Text( value='custom_experiment', placeholder='Enter experiment name', description='Name:', disabled=False)

topology_selector = widgets.Dropdown(options=['[Keep original]'] + os.listdir("topologies"), description='Topology:')
topology_upload = widgets.FileUpload(accept='.json', multiple=False, description='Upload Topology')
topology_row = widgets.HBox([topology_selector, topology_upload])

workload_selector = widgets.Dropdown(options=['[Keep original]'] + os.listdir("workload_traces"), description='Workload:')
workload_upload = widgets.FileUpload(multiple=False, description='Upload Workload')
workload_row = widgets.HBox([workload_selector, workload_upload])

failure_selector = widgets.Dropdown(options=['[Keep original]'] + os.listdir("failure_traces"), description='Failures:')
failure_upload = widgets.FileUpload(accept='.parquet', multiple=False, description='Upload Failures')
failure_row = widgets.HBox([failure_selector, failure_upload])  #Think what to do if multiple experiments need to be run

experiment_selector = widgets.Dropdown(options=os.listdir("experiments"), description='Experiment:')
experiment_upload = widgets.FileUpload(accept='.json', multiple=False, description='Upload Experiment')
experiment_row = widgets.HBox([experiment_selector, experiment_upload])

run_button = widgets.Button(description="Run Experiment")
output = widgets.Output()

def on_run_clicked(b): 
    with output:
        output.clear_output()

        if experiment_upload.value:
            experiment_file = save_uploaded_file(experiment_upload, "experiments")
            print("Experiment Uploaded successfully")
            refresh_dropdown(experiment_selector, "experiments", experiment_file, False)
        else:
            experiment_file = experiment_selector.value

        experiment_path = os.path.join("experiments", experiment_file)
        experiment = load_experiment_template(experiment_path)
        if experiment is None:
            return


        if topology_upload.value:
            topology_file = save_uploaded_file(topology_upload, "topologies")
            refresh_dropdown(topology_selector, "topologies", topology_file, True)
            topology_value = topology_file
        else:
            topology_value = topology_selector.value


        if workload_upload.value:
            workload_file = save_uploaded_file(workload_upload, "workload_traces")
            refresh_dropdown(workload_selector, "workload_traces", workload_file, True)
            workload_value = workload_file
        else:
            workload_value = workload_selector.value

        
        if failure_upload.value:
            failure_file = save_uploaded_file(failure_upload, "failure_traces")
            refresh_dropdown(failure_selector, "failure_traces", failure_file, True)
            failure_value = failure_file
        else:
            failure_value = failure_selector.value

        
        selections = {
            "experiment_name": name_selector.value,
            "topology": topology_value,
            "workload": workload_value,
            "failures": failure_value
        }

        updated_experiment = update_experiment_fields(experiment, selections)

        filename = name_selector.value
        save_json_file(updated_experiment, filename, "experiments/")
        save_path = os.path.join("experiments", f"{filename}.json")
        run_experiment(save_path)



run_button.on_click(on_run_clicked)

display(
    widgets.VBox([
        widgets.HTML("<b>OpenDC Experiment Configurator</b>"),
        name_selector,
        experiment_row,
        topology_row,
        workload_row,
        failure_row,
        run_button,
        output
    ])
)

#TO-DO ADD EXPORT POSSIBILITY
#For Dante - figure out why policies are not working,
# ask if the approach is fine,
# ask what to add more and structure, 
# ask how to export it, is zip fine or there could be Java problems (like I had) hence Docker?
# ask what to do with different types, how to not hardcode them


# Customizability

In [2]:
#TO-DO make it possible to change one or multiple parameters from each file
#TO-DO make it generate files based on provided range
#TO-DO fix output file naming
#TO-DO 

# Exporting the capsule

In [ ]:
# TO-DO ask Dante if ZIP would be fine if we have scripts to run for windows and Linux
# TO-DO ask Dante if we should zip all of the files or just selected and how to deal with multiple experiments
# I think the same notebook should be included as well and run from there